In [7]:
import pandas as pd
import numpy as np
# Load pipe-separated file
data_path = r'C:\Users\kalki\OneDrive\Desktop\week3\insurance-risk-analytics-week3\data\MachineLearningRating_v3.csv'
df = pd.read_csv(data_path, sep='|')

# Quick look at data

df.head()


C:\Users\kalki\AppData\Local\Temp\ipykernel_33440\514320426.py:5: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path, sep='|')


,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


2. Select Your Metrics

In [10]:
# 1 = policy has a claim, 0 = no claim
df["HasClaim"] = np.where(df["TotalClaims"] > 0, 1, 0)

# Claim Frequency = proportion of policies with at least one claim
claim_frequency = df["HasClaim"].mean()
print("Overall Claim Frequency:", claim_frequency)

# Claim Severity = average claim amount for policies with a claim
df["ClaimSeverity"] = df.loc[df["HasClaim"]==1, "TotalClaims"]
average_severity = df["ClaimSeverity"].mean()
print("Average Claim Severity:", average_severity)

# Margin = TotalPremium - TotalClaims
df["Margin"] = df["TotalPremium"] - df["TotalClaims"]
print("Overall Margin:", df["Margin"].sum())

Overall Claim Frequency: 0.002787726802773328
Average Claim Severity: 23273.387063228372
Overall Margin: -2955983.474269499


3 — Data Segmentation

In [11]:
# Example 1: Segment by Gender
group_a_gender = df[df["Gender"] == "Female"]  # Control group
group_b_gender = df[df["Gender"] == "Male"]    # Test group

# Example 2: Segment by Province (Gauteng vs Western Cape)
group_a_province = df[df["Province"] == "Gauteng"]
group_b_province = df[df["Province"] == "Western Cape"]

# Example 3: Segment by ZipCode (choose two representative codes)
zip_codes = df["PostalCode"].unique()[:2]  # take first two as example
group_a_zip = df[df["PostalCode"] == zip_codes[0]]
group_b_zip = df[df["PostalCode"] == zip_codes[1]]

# Check basic statistics for groups
def group_stats(group, name):
    print(f"\n{name} Stats:")
    print("Claim Frequency:", group["HasClaim"].mean())
    print("Average Claim Severity:", group.loc[group["HasClaim"]==1, "ClaimSeverity"].mean())
    print("Average Margin:", group["Margin"].mean())

group_stats(group_a_gender, "Female")
group_stats(group_b_gender, "Male")
group_stats(group_a_province, "Gauteng")
group_stats(group_b_province, "Western Cape")
group_stats(group_a_zip, f"Zip {zip_codes[0]}")
group_stats(group_b_zip, f"Zip {zip_codes[1]}")



Female Stats:
Claim Frequency: 0.002072538860103627
Average Claim Severity: 17874.72130325815
Average Margin: 8.028786746001918

Male Stats:
Claim Frequency: 0.0021953896816684962
Average Claim Severity: 14858.552293766337
Average Margin: 4.2842534672673

Gauteng Stats:
Claim Frequency: 0.0033564800121869166
Average Claim Severity: 22243.878395705604
Average Margin: -13.558894276343386

Western Cape Stats:
Claim Frequency: 0.002166327080259491
Average Claim Severity: 28095.849880512094
Average Margin: -3.4146889132270224

Zip 1459 Stats:
Claim Frequency: 0.0
Average Claim Severity: nan
Average Margin: 73.86583768017417

Zip 1513 Stats:
Claim Frequency: 0.0
Average Claim Severity: nan
Average Margin: 83.83098056271676


4 — Statistical Testing

Gender Comparison

In [12]:
from scipy.stats import chi2_contingency, ttest_ind

# Chi-square test for Claim Frequency by Gender
contingency_gender = pd.crosstab(df["Gender"], df["HasClaim"])
chi2_gender, p_freq_gender, dof, expected = chi2_contingency(contingency_gender)
print("Gender Claim Frequency p-value:", p_freq_gender)

# T-test for Claim Severity by Gender (only for claims)
severity_female = df[df["Gender"]=="Female"]["ClaimSeverity"].dropna()
severity_male = df[df["Gender"]=="Male"]["ClaimSeverity"].dropna()
t_stat_sev_gender, p_sev_gender = ttest_ind(severity_female, severity_male, equal_var=False)
print("Gender Claim Severity p-value:", p_sev_gender)

# T-test for Margin by Gender
margin_female = df[df["Gender"]=="Female"]["Margin"]
margin_male = df[df["Gender"]=="Male"]["Margin"]
t_stat_margin_gender, p_margin_gender = ttest_ind(margin_female, margin_male, equal_var=False)
print("Gender Margin p-value:", p_margin_gender)


Gender Claim Frequency p-value: 0.026570248768437145
Gender Claim Severity p-value: 0.5680286951630678
Gender Margin p-value: 0.8015464193501282


Province Comparison

In [13]:
# Claim Frequency
contingency_prov = pd.crosstab(df["Province"], df["HasClaim"])
chi2_prov, p_freq_prov, dof, expected = chi2_contingency(contingency_prov)
print("Province Claim Frequency p-value:", p_freq_prov)

# Claim Severity (only for claims)
severity_gauteng = df[df["Province"]=="Gauteng"]["ClaimSeverity"].dropna()
severity_wc = df[df["Province"]=="Western Cape"]["ClaimSeverity"].dropna()
t_stat_sev_prov, p_sev_prov = ttest_ind(severity_gauteng, severity_wc, equal_var=False)
print("Province Claim Severity p-value:", p_sev_prov)

# Margin
margin_gauteng = df[df["Province"]=="Gauteng"]["Margin"]
margin_wc = df[df["Province"]=="Western Cape"]["Margin"]
t_stat_margin_prov, p_margin_prov = ttest_ind(margin_gauteng, margin_wc, equal_var=False)
print("Province Margin p-value:", p_margin_prov)


Province Claim Frequency p-value: 5.925510718204678e-19
Province Claim Severity p-value: 0.03059896049292002
Province Margin p-value: 0.16359633510119226


In [14]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, ttest_ind

# Load your data if not already loaded
data_path = r'C:\Users\kalki\OneDrive\Desktop\week3\insurance-risk-analytics-week3\data\MachineLearningRating_v3.csv'
df = pd.read_csv(data_path, sep='|')

# Step 2 metrics (if not already created)
df["HasClaim"] = np.where(df["TotalClaims"] > 0, 1, 0)
df["ClaimSeverity"] = df["TotalClaims"] / df["HasClaim"].replace(0, np.nan)
df["Margin"] = df["TotalPremium"] - df["TotalClaims"]

# Define tests function
def run_tests(group_col, numeric_cols=["ClaimSeverity", "Margin"]):
    results = []
    
    # Chi-square test for Claim Frequency
    contingency = pd.crosstab(df[group_col], df["HasClaim"])
    chi2, p_freq, dof, expected = chi2_contingency(contingency)
    freq_result = "Reject H0" if p_freq < 0.05 else "Fail to Reject H0"
    
    # T-tests for numerical metrics
    groups = df[group_col].dropna().unique()
    if len(groups) >= 2:
        group1 = df[df[group_col]==groups[0]]
        group2 = df[df[group_col]==groups[1]]
        
        for num_col in numeric_cols:
            data1 = group1[num_col].dropna()
            data2 = group2[num_col].dropna()
            if len(data1) > 1 and len(data2) > 1:
                t_stat, p_val = ttest_ind(data1, data2, equal_var=False)
                result = "Reject H0" if p_val < 0.05 else "Fail to Reject H0"
                results.append({
                    "Feature": group_col,
                    "Metric": num_col,
                    "p-value": p_val,
                    "Conclusion": result
                })
    
    # Add Claim Frequency result
    results.append({
        "Feature": group_col,
        "Metric": "ClaimFrequency",
        "p-value": p_freq,
        "Conclusion": freq_result
    })
    
    return results

# Run tests for the required features
all_results = []
features_to_test = ["Gender", "Province", "PostalCode"]  # Adjust postal codes as needed
for feat in features_to_test:
    all_results.extend(run_tests(feat))

# Convert to DataFrame for nice display
results_df = pd.DataFrame(all_results)
print(results_df)

# Placeholder for business interpretation
for idx, row in results_df.iterrows():
    if row["Conclusion"] == "Reject H0":
        print(f"Business Insight: Significant difference detected in {row['Metric']} across {row['Feature']}. Recommend adjusting strategy accordingly.")
    else:
        print(f"Business Insight: No significant difference detected in {row['Metric']} across {row['Feature']}.")


C:\Users\kalki\AppData\Local\Temp\ipykernel_33440\3264867023.py:7: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path, sep='|')


      Feature          Metric       p-value         Conclusion
0      Gender   ClaimSeverity  2.449821e-03          Reject H0
1      Gender          Margin  2.652864e-01  Fail to Reject H0
2      Gender  ClaimFrequency  2.657025e-02          Reject H0
3    Province   ClaimSeverity  6.558100e-04          Reject H0
4    Province          Margin  3.530213e-01  Fail to Reject H0
5    Province  ClaimFrequency  5.925511e-19          Reject H0
6  PostalCode          Margin  6.630316e-01  Fail to Reject H0
7  PostalCode  ClaimFrequency  3.152172e-30          Reject H0
Business Insight: Significant difference detected in ClaimSeverity across Gender. Recommend adjusting strategy accordingly.
Business Insight: No significant difference detected in Margin across Gender.
Business Insight: Significant difference detected in ClaimFrequency across Gender. Recommend adjusting strategy accordingly.
Business Insight: Significant difference detected in ClaimSeverity across Province. Recommend adjusting st

## Task 3: Hypothesis Testing – Key Findings

We performed A/B hypothesis testing to statistically validate differences in insurance risk and profitability across key segments: **Gender, Province, and PostalCode**. The metrics analyzed were **Claim Frequency**, **Claim Severity**, and **Margin**.

### Summary of Results

| Feature    | Metric         | p-value  | Conclusion                                    |
| ---------- | -------------- | -------- | --------------------------------------------- |
| Gender     | ClaimSeverity  | 0.00245  | Reject H₀ – significant difference            |
| Gender     | Margin         | 0.2653   | Fail to Reject H₀ – no significant difference |
| Gender     | ClaimFrequency | 0.0266   | Reject H₀ – significant difference            |
| Province   | ClaimSeverity  | 0.00066  | Reject H₀ – significant difference            |
| Province   | Margin         | 0.3530   | Fail to Reject H₀ – no significant difference |
| Province   | ClaimFrequency | 5.93e-19 | Reject H₀ – significant difference            |
| PostalCode | Margin         | 0.6630   | Fail to Reject H₀ – no significant difference |
| PostalCode | ClaimFrequency | 3.15e-30 | Reject H₀ – significant difference            |

### Interpretation & Business Insights

* **Gender**: Women have a statistically lower claim frequency and different claim severity than men, indicating a need for gender-aware risk adjustments. Margin differences are not significant.
* **Province**: Claim frequency and severity vary significantly across provinces, particularly between Gauteng and Western Cape. Premiums may need regional adjustments. Margin differences remain insignificant.
* **PostalCode**: Certain postal codes show highly significant differences in claim frequency, but margin differences are not significant. Geographic targeting could improve risk management.

**Next Steps:**

* Use these validated risk differences to inform a new **segmentation strategy**.
* Consider **differential pricing or risk-based product offerings** for segments showing statistically significant differences in claims.
* Incorporate additional features and A/B tests for finer-grained segmentation in future analyses.

---